## Matrix sparsity

A common challenge with real-world ratings data is that most users will not have rated most items, and most items will only have been rated by a small number of users. This results in a very empty or sparse DataFrame.

In this exercise, you will calculate how sparse the movie_lens ratings data is by counting the number of occupied cells and compare it to the size of the full DataFrame. The DataFrame user_ratings_df that you have used in previous exercises, containing a row per user and a column per movie, has been loaded for you.

### Instructions
    - Count the number of non-empty cells in user_ratings_df and store the result as sparsity_count.
    - Count the total number of cells in the user_ratings_df DataFrame and store it as full_count.
    - Calculate the sparsity of the DataFrame by dividing the number of non-empty cells by the total number of cells and print the result.

In [ ]:
# Count the occupied cells
sparsity_count = user_ratings_df.isnull().values.sum()

# Count all cells
full_count = user_ratings_df.size

# Find the sparsity of the DataFrame
sparsity = sparsity_count / full_count
print(sparsity)

## Limited data in your rows

This data sparsity can cause an issue when using techniques like K-nearest neighbors as discussed in the last chapter. KNN needs to find the k most similar users that have rated an item, but if only less than or equal to k users have given an item the rating, all ratings will be the "most similar".

In this exercise, you will count how often each movie in the user_ratings_df DataFrame has been given a rating, and then see how many have only one or two ratings.

### Instructions 1/3
    - Count the number of non-empty cells in each column of user_ratings_df and store it as occupied_count.

In [ ]:
# Count the occupied cells per column
occupied_count = user_ratings_df.notnull().sum()
print(occupied_count)

### Instructions 2/3
    - Sort occupied_count from low to high. Looking at the resulting sorted Series, note the number of movies with one review.

In [ ]:
# Count the occupied cells per column
occupied_count = user_ratings_df.notnull().sum()

# Sort the resulting series from low to high
sorted_occupied_count = occupied_count.sort_values()
print(sorted_occupied_count)

### Instructions 3/3
    - Create a histogram of the sorted_occupied_count Series you just created. matplotlib.pyplothas been loaded as plt.

In [ ]:
# Count the occupied cells per column
occupied_count = user_ratings_df.notnull().sum()

# Sort the resulting series from low to high
sorted_occupied_count = occupied_count.sort_values()

# Plot a histogram of the values in sorted_occupied_count
sorted_occupied_count.hist()
plt.show()

## Information loss in factorization

You may wonder how the factors with far fewer columns can summarize a larger DataFrame without loss. In fact, it doesn't — the factors we create are generally a close approximation of the data, as it is inevitable for some information to be lost. This means that predicted values might not be exact, but should be close enough to be useful.

In this exercise, you will inspect the same original pre-factorization DataFrame from the last exercise loaded as original_df, and compare it to the product of its two factors, user_matrix and item_matrix.

### Instructions
    - Find the dot product of user_matrix and item_matrix and store it as predictions_df.

In [ ]:
import numpy as np

# Multiply the user and item matrices
predictions_df = np.dot(user_matrix, item_matrix)
# Inspect the recreated DataFrame
print(predictions_df)

# Inspect the original DataFrame and compare
print(original_df)

## Normalize your data

Before you can find the factors of the ratings matrix using singular value decomposition, you will need to "de-mean", or center it, by subtracting each row's mean from each value in that row.

In this exercise, you will begin prepping the movie rating DataFrame you have been working with in order to be able to perform Singular value decomposition.

user_ratings_df contains a row per user and a column for each movie and has been loaded for you.

### Instructions
    - Find the average rating each user has given across all the movies they have seen and store these values as avg_ratings.
    - Subtract the row averages from their respective rows and store the result as user_ratings_centered.
    - Finally, fill in all missing values in user_ratings_centered with zeros.
    - Print the average of each column in user_ratings_centered to show they have been de-meaned.

In [ ]:
# Get the average rating for each user 
avg_ratings = user_ratings_df.mean(axis=1)

# Center each user's ratings around 0
user_ratings_centered = user_ratings_df.sub(avg_ratings, axis=1)

# Fill in all missing values with 0s
user_ratings_centered.fillna(0, inplace=True)

# Print the mean of each column
print(user_ratings_centered.mean(axis=1))

## Decomposing your matrix

Now that you have prepped your data by centering it and filling in the remaining empty values with 0, you can get around to finding your data's factors. In this exercise, you will break the user_ratings_centered data you generated in the last exercise into 3 factors: U, sigma, and Vt.

    - U is a matrix with a row for each user
    - Vt has a column for each movie
    - sigma is an array of weights that you will need to convert to a diagonal matrix

The user_ratings_centered that you created in the last lesson has been loaded for you.

### Instructions 1/2
    - Import svds from scipy.sparse.linalg.
    - Decompose user_ratings_pivot_centered into its factor matrices: U, sigma and Vt.

In [ ]:
# Import the required libraries 
from scipy.sparse.linalg import svds
import numpy as np

# Decompose the matrix
U, sigma, Vt = svds(user_ratings_centered)

### Instructions 2/2
    - Convert the sigma array into a diagonal matrix.

In [ ]:
# Import the required libraries 
from scipy.sparse.linalg import svds
import numpy as np

# Decompose the matrix
U, sigma, Vt = svds(user_ratings_centered)

# Convert sigma into a diagonal matrix
sigma = np.diag(sigma)
print(sigma)

## Recalculating the matrix

Now that you have your three factor matrices, you can multiply them back together to get complete ratings data without missing values. In this exercise, you will use numpy's dot product function to multiply U and sigma first, then the result by Vt. You will then be able add the average ratings for each row to find your final ratings.

U, sigma, Vt, avg_ratings, and user_ratings_df from the previous exercise have been loaded for you. Also, numpy has been loaded as np.

### Instructions 1/4
    - Find the dot product of the matrix U and sigma.

In [ ]:
# Dot product of U and sigma
U_sigma = np.dot(U, sigma)

### Instructions 2/4
    - Find the dot product of U_sigma and Vt and print the result.

In [ ]:
# Dot product of U and sigma
U_sigma = np.dot(U, sigma)

# Dot product of result and Vt
U_sigma_Vt = np.dot(U_sigma, Vt)

# Print the result
print(U_sigma_Vt)

### Instructions 3/4
    - Reshape the values of avg_ratings and add them back onto U_sigma_Vt.

In [ ]:
# Dot product of U and sigma
U_sigma = np.dot(U, sigma)

# Dot product of result and Vt
U_sigma_Vt = np.dot(U_sigma, Vt)

# Add the row means back contained in avg_ratings
uncentered_ratings = U_sigma_Vt + avg_ratings.values.reshape(-1, 1)

### Instructions 4/4
    - Create a DataFrame of the results using the original index and column names from user_ratings_df.